# CPS600 - Python Programming for Finance 
###  
<img src="https://www.syracuse.edu/wp-content/themes/g6-carbon/img/syracuse-university-seal.svg?ver=6.3.9" style="width: 200px;"/>

# Performance Python (Lab)

###  September 18, 2018

# Performance Python

## Python Paradigms and Performance

In [11]:
def perf_comp_data(func_list, data_list, rep=3, number=1):
    ''' Function to compare the performance of different functions.
    
    Parameters
    ==========
    func_list : list
        list with function names as strings
    data_list : list
        list with data set names as strings
    rep : int
        number of repetitions of the whole comparison
    number : int
        number of executions for every function
    '''
    from timeit import repeat
    res_list = {}
    for name in enumerate(func_list):
        stmt = name[1] + '(' + data_list[name[0]] + ')'
        setup = "from __main__ import " + name[1] + ', '+ data_list[name[0]]
        results = repeat(stmt=stmt, setup=setup,
                         repeat=rep, number=number)
        res_list[name[1]] = sum(results) / rep
    res_sort = sorted(res_list.items(),
                      key=lambda x: (x[1], x[0]))
    for item in res_sort:
        rel = item[1] / res_sort[0][1]
        print ('function: ' + item[0] +
              ', av. time sec: %9.5f, ' % item[1]
            + 'relative: %6.1f' % rel)

#### We want to evaluate a somewhat complex mathematical expression on an array with 500,000 numbers.

$\Large{y = \sqrt{|\cos(x)|}  +  \sin(2 + 3x)}$

In [ ]:
from math import *
def f(x):
    return abs(cos(x)) ** 0.5 + sin(2 + 3 * x)

In [ ]:
# generate  a list object with 500,000 numbers that we can work with

I = 500000
a_py = range(I)

In [ ]:
# f1 loops over the whole data set and appends 
# the single results of the function evaluations to a results list object

def f1(a):
    res = []
    for x in a:
        res.append(f(x))
    return res

In [ ]:
# f2 is using a list comprehension

def f2(a):
    return [f(x) for x in a]

In [ ]:
# f3 uses eval function and list comprehension

def f3(a):
    ex = 'abs(cos(x)) ** 0.5 + sin(2 + 3 * x)'
    return [eval(ex) for x in a]

In [ ]:
# the array of data is an ndarray object instead of a list object

import numpy as np
a_np = np.arange(I)

In [ ]:
# f4 use of NumPy vectorization techniques

def f4(a):
    return (np.abs(np.cos(a)) ** 0.5 +
            np.sin(2 + 3 * a))

In [ ]:
import numexpr as ne

In [ ]:
# f5 numexpr to evaluate the numerical expression using single thread
def f5(a):
    ex = 'abs(cos(a)) ** 0.5 + sin(2 + 3 * a)'
    ne.set_num_threads(1)
    return ne.evaluate(ex)

In [ ]:
# f6 numexpr to evaluate the numerical expression using 4 threads

def f6(a):
    ex = 'abs(cos(a)) ** 0.5 + sin(2 + 3 * a)'
    ne.set_num_threads(4)
    return ne.evaluate(ex)

In [ ]:
%%time
r1 = f1(a_py)
r2 = f2(a_py)
r3 = f3(a_py)
r4 = f4(a_np)
r5 = f5(a_np)
r6 = f6(a_np)
# WAIT 25 sec
# IPython cell magic command %%time records the total execution time

In [ ]:
# The NumPy function allclose allows to check whether two ndarray(-like) objects contain the same data

np.allclose(r1, r2)

In [ ]:
np.allclose(r1, r3)

In [ ]:
np.allclose(r1, r4)

In [ ]:
np.allclose(r1, r5)

In [ ]:
np.allclose(r1, r6)

In [ ]:
func_list = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']
data_list = ['a_py', 'a_py', 'a_py', 'a_np', 'a_np', 'a_np']

In [ ]:
# how the different implementations compare with respect to execution speed.

perf_comp_data(func_list, data_list, rep=1)
# WAIT 1 min

##  Sequential vs. Parallel Computing

### The Monte Carlo Algorithm

####  Black-Scholes-Merton SDE

$\Large{dS_t = r S_t dt + \sigma S_t dZ_t}$

#### Monte Carlo estimator for European call option

$\Large{C_0 = e^{-rT} \frac{1}{I} \displaystyle\sum_I max(S_T (i) - K, 0)}$

In [19]:
def bsm_mcs_valuation(strike):
    ''' Dynamic Black-Scholes-Merton Monte Carlo estimator
    for European calls.
    
    Parameters
    ==========
    strike : float
        strike price of the option
    
    Results
    =======
    value : float
        estimate for present value of call option
    '''
    import numpy as np
    S0 = 100.; T = 1.0; r = 0.05; vola = 0.2
    M = 50; I = 20000
    dt = T / M
    rand = np.random.standard_normal((M + 1, I))
    S = np.zeros((M + 1, I)); S[0] = S0
    for t in range(1, M + 1):
        S[t] = S[t-1] * np.exp((r - 0.5 * vola ** 2) * dt
                               + vola * np.sqrt(dt) * rand[t])
    value = (np.exp(-r * T)
                     * np.sum(np.maximum(S[-1] - strike, 0)) / I)
    return value

### The Sequential Calculation

In [ ]:
# The function seq_value calculates the Monte Carlo estimators 
# and returns list objects containing strikes and valuation results

def seq_value(n):
    ''' Sequential option valuation.
    
    Parameters
    ==========
    n : int
        number of option valuations/strikes
    '''
    strikes = np.linspace(80, 120, n)
    option_values = []
    for strike in strikes:
        option_values.append(bsm_mcs_valuation(strike))
    return strikes, option_values

In [ ]:
import numpy as np
n = 100  # number of options to be valued
%time strikes, option_values_seq = seq_value(n)
# WAIT 10 sec

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(8, 4))
plt.plot(strikes, option_values_seq, 'b')
plt.plot(strikes, option_values_seq, 'r.')
plt.grid(True)
plt.xlabel('strikes')
plt.ylabel('European call option values')
plt.title("European call option values by Monte Carlo simulation")
# size: 60

### The Parallel Calculation - SKIP
* missing: from IPython.parallel import Client
* missing: from ipyparallel import Client

In [51]:
# THIS DOES NOT WORK 

# CONDA INSTALL IPYPARALLEL
# This require cluster

#from IPython.parallel import Client
# from parallelpy import Client
from ipyparallel import Client

c = Client(profile="default")
view = c.load_balanced_view()

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json


OSError: Connection file '~/.ipython/profile_default/security/ipcontroller-client.json' not found.
You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.

In [ ]:
def par_value(n):
    ''' Sequential option valuation.
    
    Parameters
    ==========
    n : int
        number of option valuations/strikes
    '''
    strikes = np.linspace(80, 120, n)
    option_values = []
    for strike in strikes:
        value = view.apply_async(bsm_mcs_valuation, strike)
        option_values.append(value)
    c.wait(option_values)
    return strikes, option_values

In [ ]:
%time strikes, option_values_obj = par_value(n)

In [ ]:
option_values_obj[0].metadata

In [ ]:
option_values_obj[0].result

In [ ]:
option_values_par = []
for res in option_values_obj:
    option_values_par.append(res.result)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(strikes, option_values_seq, 'b', label='Sequential')
plt.plot(strikes, option_values_par, 'r.', label='Parallel')
plt.grid(True); plt.legend(loc=0)
plt.xlabel('strikes')
plt.ylabel('European call option values')
# tag: option_comp
# title: Comparison of European call option values
# size: 60

### Performance Comparison - SKIP

In [ ]:
# SKIP BECAUSE PAR_VALUE CANNOT WORK

n = 25  # number of option valuations
func_list = ['seq_value', 'par_value']
data_list = 2 * ['n']

In [ ]:
perf_comp_data(func_list, data_list, rep=3)

## MultiProcessing  - SKIP 

In [ ]:
import multiprocessing as mp
import numpy as np

In [ ]:
import math
def simulate_geometric_brownian_motion(p):
    M, I = p
      # time steps, paths
    S0 = 100; r = 0.05; sigma = 0.2; T = 1.0
      # model parameters
    dt = T / M
    paths = np.zeros((M + 1, I))
    paths[0] = S0
    for t in range(1, M + 1):
        paths[t] = paths[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt +
                    sigma * math.sqrt(dt) * np.random.standard_normal(I))
    return paths

In [ ]:
paths = simulate_geometric_brownian_motion((5, 2))
paths

In [ ]:
I = 10000  # number of paths
M = 50  # number of time steps
t = 20  # number of tasks/simulations

In [ ]:
# running on server with 4 cores
from time import time
times = []
for w in range(1, 5):
    t0 = time()
    pool = mp.Pool(processes=w)
      # the pool of workers
    result = pool.map(simulate_geometric_brownian_motion, t * [(M, I), ])
      # the mapping of the function to the list of parameter tuples
    times.append(time() - t0)


In [ ]:
plt.plot(range(1, 5), times)
plt.plot(range(1, 5), times, 'ro')
plt.grid(True)
plt.xlabel('number of processes')
plt.ylabel('time in seconds')
plt.title('%d Monte Carlo simulations' % t)
# tag: multi_proc
# title: Comparison execution speed dependent on the number of threads used (4 core machine)
# size: 60

## Dynamic Compiling

### Introductory Example

In [1]:
from math import cos, log
def f_py(I, J):
    res = 0
    for i in range(I):
        for j in range (J):
            res += int(cos(log(1)))
    return res

In [2]:
I, J = 5000, 5000
%time f_py(I, J)

Wall time: 26.2 s


25000000

In [3]:
import numpy as np
def f_np(I, J):
    a = np.ones((I, J), dtype=np.float64)
    return int(np.sum(np.cos(np.log(a)))), a

In [4]:
%time res, a = f_np(I, J)

Wall time: 1.04 s


In [5]:
a.nbytes

200000000

In [6]:
import numba as nb
# WAIT several seconds

In [7]:
f_nb = nb.jit(f_py)

In [8]:
%time f_nb(I, J)

Wall time: 2.86 s


25000000

In [9]:
func_list = ['f_py', 'f_np', 'f_nb']
data_list = 3 * ['I, J']

In [12]:
perf_comp_data(func_list, data_list, rep=3)
#WAIT 1 min

function: f_nb, av. time sec:   0.00000, relative:    1.0
function: f_np, av. time sec:   0.99355, relative: 222897.0
function: f_py, av. time sec:  25.94344, relative: 5820253.6


### Binomial Option Pricing

In [15]:
import numpy as np
# model & option Parameters
S0 = 100.  # initial index level
T = 1.  # call option maturity
r = 0.05  # constant short rate
vola = 0.20  # constant volatility factor of diffusion

# time parameters
M = 1000  # time steps
dt = T / M  # length of time interval
df = np.exp(-r * dt)  # discount factor per time interval

# binomial parameters
u = np.exp(vola * np.sqrt(dt))  # up-movement
d = 1 / u  # down-movement
q = (np.exp(r * dt) - d) / (u - d)  # martingale probability

In [20]:
import numpy as np
def binomial_py(strike):
    ''' Binomial option pricing via looping.
    
    Parameters
    ==========
    strike : float
        strike price of the European call option
    '''
    # LOOP 1 - Index Levels
    S = np.zeros((M + 1, M + 1), dtype=np.float64)
      # index level array
    S[0, 0] = S0
    z1 = 0
    for j in range(1, M + 1, 1):
        z1 = z1 + 1
        for i in range(z1 + 1):
            S[i, j] = S[0, 0] * (u ** j) * (d ** (i * 2))
            
    # LOOP 2 - Inner Values
    iv = np.zeros((M + 1, M + 1), dtype=np.float64)
      # inner value array
    z2 = 0
    for j in range(0, M + 1, 1):
        for i in range(z2 + 1):
            iv[i, j] = max(S[i, j] - strike, 0)
        z2 = z2 + 1
        
    # LOOP 3 - Valuation
    pv = np.zeros((M + 1, M + 1), dtype=np.float64)
      # present value array
    pv[:, M] = iv[:, M]  # initialize last time point
    z3 = M + 1
    for j in range(M - 1, -1, -1):
        z3 = z3 - 1
        for i in range(z3):
            pv[i, j] = (q * pv[i, j + 1] +
                        (1 - q) * pv[i + 1, j + 1]) * df
    return pv[0, 0]

In [17]:
%time round(binomial_py(100), 3)

Wall time: 2.95 s


10.449

In [21]:
%time round(bsm_mcs_valuation(100), 3)

Wall time: 161 ms


10.375

In [22]:
def binomial_np(strike):
    ''' Binomial option pricing with NumPy.
    
    Parameters
    ==========
    strike : float
        strike price of the European call option
    '''
    # Index Levels with NumPy
    mu = np.arange(M + 1)
    mu = np.resize(mu, (M + 1, M + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = S0 * mu * md
    
    # Valuation Loop
    pv = np.maximum(S - strike, 0)
    qu = np.zeros((M + 1, M + 1), dtype=np.float64)
    qu[:, :] = q  
    qd = 1 - qu 
    z = 0
    for t in range(M - 1, -1, -1):  # backwards iteration
        pv[0:M - z, t] = (qu[0:M - z, t] * pv[0:M - z, t + 1]
                        + qd[0:M - z, t] * pv[1:M - z + 1, t + 1]) * df
        z += 1
    return pv[0, 0]

In [23]:
M = 4  # four time steps only
mu = np.arange(M + 1)
mu

array([0, 1, 2, 3, 4])

In [24]:
mu = np.resize(mu, (M + 1, M + 1))
mu

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [25]:
md = np.transpose(mu)
md

array([[0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4]])

In [26]:
mu = u ** (mu - md)
mu.round(3)

array([[ 1.   ,  1.006,  1.013,  1.019,  1.026],
       [ 0.994,  1.   ,  1.006,  1.013,  1.019],
       [ 0.987,  0.994,  1.   ,  1.006,  1.013],
       [ 0.981,  0.987,  0.994,  1.   ,  1.006],
       [ 0.975,  0.981,  0.987,  0.994,  1.   ]])

In [27]:
md = d ** md
md.round(3)

array([[ 1.   ,  1.   ,  1.   ,  1.   ,  1.   ],
       [ 0.994,  0.994,  0.994,  0.994,  0.994],
       [ 0.987,  0.987,  0.987,  0.987,  0.987],
       [ 0.981,  0.981,  0.981,  0.981,  0.981],
       [ 0.975,  0.975,  0.975,  0.975,  0.975]])

In [28]:
S = S0 * mu * md
S.round(3)

array([[ 100.   ,  100.634,  101.273,  101.915,  102.562],
       [  98.743,   99.37 ,  100.   ,  100.634,  101.273],
       [  97.502,   98.121,   98.743,   99.37 ,  100.   ],
       [  96.276,   96.887,   97.502,   98.121,   98.743],
       [  95.066,   95.669,   96.276,   96.887,   97.502]])

In [29]:
M = 1000  # reset number of time steps
%time round(binomial_np(100), 3)

Wall time: 388 ms


10.449

In [30]:
binomial_nb = nb.jit(binomial_py)

In [31]:
%time round(binomial_nb(100), 3)

Wall time: 980 ms


10.449

In [32]:
func_list = ['binomial_py', 'binomial_np', 'binomial_nb']
K = 100.
data_list = 3 * ['K']

In [33]:
perf_comp_data(func_list, data_list, rep=1)

function: binomial_np, av. time sec:   0.42617, relative:    1.0
function: binomial_nb, av. time sec:   0.78862, relative:    1.9
function: binomial_py, av. time sec:   2.93673, relative:    6.9


## Static Compiling with Cython - SKIP
* missing:   from nested_loop import f_cy

In [34]:
def f_py(I, J):
    res = 0.  # we work on a float object
    for i in range(I):
        for j in range (J * I):
            res += 1
    return res

In [35]:
I, J = 500, 500
%time f_py(I, J)

Wall time: 24.3 s


125000000.0

In [36]:
import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x2788062b0b8>)

In [37]:
# DOES NOT WORK !!!!!!!!!
# REST IN THIS SECTION IS SKIPPED

from nested_loop import f_cy

ImportError: No module named 'nested_loop'

In [ ]:
%time res = f_cy(I, J)

In [ ]:
res

In [ ]:
%load_ext cythonmagic

In [ ]:
%%cython
#
# Nested loop example with Cython
#
def f_cy(int I, int J):
    cdef double res = 0
    # double float much slower than int or long
    for i in range(I):
        for j in range (J * I):
            res += 1
    return res

In [ ]:
%time res = f_cy(I, J)

In [ ]:
res

In [38]:
import numba as nb

In [40]:
f_nb = nb.jit(f_py)

In [41]:
%time res = f_nb(I, J)

Wall time: 459 ms


In [42]:
res

125000000.0

In [43]:
func_list = ['f_py', 'f_cy', 'f_nb']
I, J = 500, 500
data_list = 3 * ['I, J']

In [44]:
perf_comp_data(func_list, data_list, rep=1)

ImportError: cannot import name 'f_cy'

In [ ]:
# Performance Python

## Python Paradigms and Performance

def perf_comp_data(func_list, data_list, rep=3, number=1):
    ''' Function to compare the performance of different functions.
    
    Parameters
    ==========
    func_list : list
        list with function names as strings
    data_list : list
        list with data set names as strings
    rep : int
        number of repetitions of the whole comparison
    number : int
        number of executions for every function
    '''
    from timeit import repeat
    res_list = {}
    for name in enumerate(func_list):
        stmt = name[1] + '(' + data_list[name[0]] + ')'
        setup = "from __main__ import " + name[1] + ', '+ data_list[name[0]]
        results = repeat(stmt=stmt, setup=setup,
                         repeat=rep, number=number)
        res_list[name[1]] = sum(results) / rep
    res_sort = sorted(res_list.items(),
                      key=lambda x: (x[1], x[0]))
    for item in res_sort:
        rel = item[1] / res_sort[0][1]
        print ('function: ' + item[0] +
              ', av. time sec: %9.5f, ' % item[1]
            + 'relative: %6.1f' % rel)

#### We want to evaluate a somewhat complex mathematical expression on an array with 500,000 numbers.

$\Large{y = \sqrt{|\cos(x)|}  +  \sin(2 + 3x)}$

from math import *
def f(x):
    return abs(cos(x)) ** 0.5 + sin(2 + 3 * x)

# generate  a list object with 500,000 numbers that we can work with

I = 500000
a_py = range(I)

# f1 loops over the whole data set and appends 
# the single results of the function evaluations to a results list object

def f1(a):
    res = []
    for x in a:
        res.append(f(x))
    return res

# f2 is using a list comprehension

def f2(a):
    return [f(x) for x in a]

# f3 uses eval function and list comprehension

def f3(a):
    ex = 'abs(cos(x)) ** 0.5 + sin(2 + 3 * x)'
    return [eval(ex) for x in a]

# the array of data is an ndarray object instead of a list object

import numpy as np
a_np = np.arange(I)

# f4 use of NumPy vectorization techniques

def f4(a):
    return (np.abs(np.cos(a)) ** 0.5 +
            np.sin(2 + 3 * a))

import numexpr as ne

# f5 numexpr to evaluate the numerical expression using single thread
def f5(a):
    ex = 'abs(cos(a)) ** 0.5 + sin(2 + 3 * a)'
    ne.set_num_threads(1)
    return ne.evaluate(ex)

# f6 numexpr to evaluate the numerical expression using 4 threads

def f6(a):
    ex = 'abs(cos(a)) ** 0.5 + sin(2 + 3 * a)'
    ne.set_num_threads(4)
    return ne.evaluate(ex)

%%time
r1 = f1(a_py)
r2 = f2(a_py)
r3 = f3(a_py)
r4 = f4(a_np)
r5 = f5(a_np)
r6 = f6(a_np)
# WAIT 25 sec
# IPython cell magic command %%time records the total execution time

# The NumPy function allclose allows to check whether two ndarray(-like) objects contain the same data

np.allclose(r1, r2)

np.allclose(r1, r3)

np.allclose(r1, r4)

np.allclose(r1, r5)

np.allclose(r1, r6)

func_list = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']
data_list = ['a_py', 'a_py', 'a_py', 'a_np', 'a_np', 'a_np']

# how the different implementations compare with respect to execution speed.

perf_comp_data(func_list, data_list, rep=1)
# WAIT 1 min

## Memory Layout and Performance

import numpy as np

# NumPy allows the specification of a so-called dtype per ndarray object
# NumPy also allows us to choose from two different memory layouts 

#    shape              either an int, a sequence of ints, or a reference to another numpy.ndarray
#    dtype (optional)   NumPy-specific basic data types for numpy.ndarray objects
#    order (optional)   The order in which to store elements in memory: C for C-like (i.e., row-wise) 
#                       or F for Fortran-like (i.e., column-wise)

np.zeros((3, 3), dtype=np.float64, order='C')

c = np.array([[ 1.,  1.,  1.],
              [ 2.,  2.,  2.],
              [ 3.,  3.,  3.]], order='C')
# C-like (i.e., row-wise) : 1s, the 2s, and the 3s are stored next to each other

f = np.array([[ 1.,  1.,  1.],
              [ 2.,  2.,  2.],
              [ 3.,  3.,  3.]], order='F')
# F-like (i.e., column-wise) : 1, 2, and 3 are next to each other in each column

# Let’s see whether the memory layout makes a difference in some way when the array is large:

x = np.random.standard_normal((3, 1500000))
C = np.array(x, order='C')
F = np.array(x, order='F')
x = 0.0

%timeit C.sum(axis=0)

# Calculating sums over the first axis is roughly two times slower than over the second axis.

%timeit C.sum(axis=1)

%timeit C.std(axis=0)

%timeit C.std(axis=1)

# F-like layout option leads to worse performance compared to the C-like layout.

%timeit F.sum(axis=0)

%timeit F.sum(axis=1)

%timeit F.std(axis=0)

%timeit F.std(axis=1)

C = 0.0; F = 0.0

##  Sequential vs. Parallel Computing

### The Monte Carlo Algorithm

####  Black-Scholes-Merton SDE

$\Large{dS_t = r S_t dt + \sigma S_t dZ_t}$

#### Monte Carlo estimator for European call option

$\Large{C_0 = e^{-rT} \frac{1}{I} \displaystyle\sum_I max(S_T (i) - K, 0)}$

def bsm_mcs_valuation(strike):
    ''' Dynamic Black-Scholes-Merton Monte Carlo estimator
    for European calls.
    
    Parameters
    ==========
    strike : float
        strike price of the option
    
    Results
    =======
    value : float
        estimate for present value of call option
    '''
    import numpy as np
    S0 = 100.; T = 1.0; r = 0.05; vola = 0.2
    M = 50; I = 20000
    dt = T / M
    rand = np.random.standard_normal((M + 1, I))
    S = np.zeros((M + 1, I)); S[0] = S0
    for t in range(1, M + 1):
        S[t] = S[t-1] * np.exp((r - 0.5 * vola ** 2) * dt
                               + vola * np.sqrt(dt) * rand[t])
    value = (np.exp(-r * T)
                     * np.sum(np.maximum(S[-1] - strike, 0)) / I)
    return value

### The Sequential Calculation

# The function seq_value calculates the Monte Carlo estimators 
# and returns list objects containing strikes and valuation results

def seq_value(n):
    ''' Sequential option valuation.
    
    Parameters
    ==========
    n : int
        number of option valuations/strikes
    '''
    strikes = np.linspace(80, 120, n)
    option_values = []
    for strike in strikes:
        option_values.append(bsm_mcs_valuation(strike))
    return strikes, option_values

import numpy as np
n = 100  # number of options to be valued
%time strikes, option_values_seq = seq_value(n)
# WAIT 10 sec

import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(8, 4))
plt.plot(strikes, option_values_seq, 'b')
plt.plot(strikes, option_values_seq, 'r.')
plt.grid(True)
plt.xlabel('strikes')
plt.ylabel('European call option values')
plt.title("European call option values by Monte Carlo simulation")
# size: 60

### The Parallel Calculation - SKIP
* missing: from IPython.parallel import Client
* missing: from ipyparallel import Client

# THIS DOES NOT WORK 

# CONDA INSTALL IPYPARALLEL
# This require cluster

#from IPython.parallel import Client
# from parallelpy import Client
from ipyparallel import Client

c = Client(profile="default")
view = c.load_balanced_view()

def par_value(n):
    ''' Sequential option valuation.
    
    Parameters
    ==========
    n : int
        number of option valuations/strikes
    '''
    strikes = np.linspace(80, 120, n)
    option_values = []
    for strike in strikes:
        value = view.apply_async(bsm_mcs_valuation, strike)
        option_values.append(value)
    c.wait(option_values)
    return strikes, option_values

%time strikes, option_values_obj = par_value(n)

option_values_obj[0].metadata

option_values_obj[0].result

option_values_par = []
for res in option_values_obj:
    option_values_par.append(res.result)

plt.figure(figsize=(8, 4))
plt.plot(strikes, option_values_seq, 'b', label='Sequential')
plt.plot(strikes, option_values_par, 'r.', label='Parallel')
plt.grid(True); plt.legend(loc=0)
plt.xlabel('strikes')
plt.ylabel('European call option values')
# tag: option_comp
# title: Comparison of European call option values
# size: 60

### Performance Comparison - SKIP

# SKIP BECAUSE PAR_VALUE CANNOT WORK

n = 25  # number of option valuations
func_list = ['seq_value', 'par_value']
data_list = 2 * ['n']

perf_comp_data(func_list, data_list, rep=3)

## MultiProcessing  - SKIP 

import multiprocessing as mp
import numpy as np

import math
def simulate_geometric_brownian_motion(p):
    M, I = p
      # time steps, paths
    S0 = 100; r = 0.05; sigma = 0.2; T = 1.0
      # model parameters
    dt = T / M
    paths = np.zeros((M + 1, I))
    paths[0] = S0
    for t in range(1, M + 1):
        paths[t] = paths[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt +
                    sigma * math.sqrt(dt) * np.random.standard_normal(I))
    return paths

paths = simulate_geometric_brownian_motion((5, 2))
paths

I = 10000  # number of paths
M = 50  # number of time steps
t = 20  # number of tasks/simulations

# running on server with 4 cores
from time import time
times = []
for w in range(1, 5):
    t0 = time()
    pool = mp.Pool(processes=w)
      # the pool of workers
    result = pool.map(simulate_geometric_brownian_motion, t * [(M, I), ])
      # the mapping of the function to the list of parameter tuples
    times.append(time() - t0)


plt.plot(range(1, 5), times)
plt.plot(range(1, 5), times, 'ro')
plt.grid(True)
plt.xlabel('number of processes')
plt.ylabel('time in seconds')
plt.title('%d Monte Carlo simulations' % t)
# tag: multi_proc
# title: Comparison execution speed dependent on the number of threads used (4 core machine)
# size: 60

## Dynamic Compiling

### Introductory Example

from math import cos, log
def f_py(I, J):
    res = 0
    for i in range(I):
        for j in range (J):
            res += int(cos(log(1)))
    return res

I, J = 5000, 5000
%time f_py(I, J)

import numpy as np
def f_np(I, J):
    a = np.ones((I, J), dtype=np.float64)
    return int(np.sum(np.cos(np.log(a)))), a

%time res, a = f_np(I, J)

a.nbytes

import numba as nb
# WAIT several seconds

f_nb = nb.jit(f_py)

%time f_nb(I, J)

func_list = ['f_py', 'f_np', 'f_nb']
data_list = 3 * ['I, J']

perf_comp_data(func_list, data_list, rep=3)
#WAIT 1 min

### Binomial Option Pricing

import numpy as np
# model & option Parameters
S0 = 100.  # initial index level
T = 1.  # call option maturity
r = 0.05  # constant short rate
vola = 0.20  # constant volatility factor of diffusion

# time parameters
M = 1000  # time steps
dt = T / M  # length of time interval
df = np.exp(-r * dt)  # discount factor per time interval

# binomial parameters
u = np.exp(vola * np.sqrt(dt))  # up-movement
d = 1 / u  # down-movement
q = (np.exp(r * dt) - d) / (u - d)  # martingale probability

import numpy as np
def binomial_py(strike):
    ''' Binomial option pricing via looping.
    
    Parameters
    ==========
    strike : float
        strike price of the European call option
    '''
    # LOOP 1 - Index Levels
    S = np.zeros((M + 1, M + 1), dtype=np.float64)
      # index level array
    S[0, 0] = S0
    z1 = 0
    for j in range(1, M + 1, 1):
        z1 = z1 + 1
        for i in range(z1 + 1):
            S[i, j] = S[0, 0] * (u ** j) * (d ** (i * 2))
            
    # LOOP 2 - Inner Values
    iv = np.zeros((M + 1, M + 1), dtype=np.float64)
      # inner value array
    z2 = 0
    for j in range(0, M + 1, 1):
        for i in range(z2 + 1):
            iv[i, j] = max(S[i, j] - strike, 0)
        z2 = z2 + 1
        
    # LOOP 3 - Valuation
    pv = np.zeros((M + 1, M + 1), dtype=np.float64)
      # present value array
    pv[:, M] = iv[:, M]  # initialize last time point
    z3 = M + 1
    for j in range(M - 1, -1, -1):
        z3 = z3 - 1
        for i in range(z3):
            pv[i, j] = (q * pv[i, j + 1] +
                        (1 - q) * pv[i + 1, j + 1]) * df
    return pv[0, 0]

%time round(binomial_py(100), 3)

%time round(bsm_mcs_valuation(100), 3)

def binomial_np(strike):
    ''' Binomial option pricing with NumPy.
    
    Parameters
    ==========
    strike : float
        strike price of the European call option
    '''
    # Index Levels with NumPy
    mu = np.arange(M + 1)
    mu = np.resize(mu, (M + 1, M + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = S0 * mu * md
    
    # Valuation Loop
    pv = np.maximum(S - strike, 0)
    qu = np.zeros((M + 1, M + 1), dtype=np.float64)
    qu[:, :] = q  
    qd = 1 - qu 
    z = 0
    for t in range(M - 1, -1, -1):  # backwards iteration
        pv[0:M - z, t] = (qu[0:M - z, t] * pv[0:M - z, t + 1]
                        + qd[0:M - z, t] * pv[1:M - z + 1, t + 1]) * df
        z += 1
    return pv[0, 0]

M = 4  # four time steps only
mu = np.arange(M + 1)
mu

mu = np.resize(mu, (M + 1, M + 1))
mu

md = np.transpose(mu)
md

mu = u ** (mu - md)
mu.round(3)

md = d ** md
md.round(3)

S = S0 * mu * md
S.round(3)

M = 1000  # reset number of time steps
%time round(binomial_np(100), 3)

binomial_nb = nb.jit(binomial_py)

%time round(binomial_nb(100), 3)

func_list = ['binomial_py', 'binomial_np', 'binomial_nb']
K = 100.
data_list = 3 * ['K']

perf_comp_data(func_list, data_list, rep=1)

## Static Compiling with Cython - SKIP
* missing:   from nested_loop import f_cy

def f_py(I, J):
    res = 0.  # we work on a float object
    for i in range(I):
        for j in range (J * I):
            res += 1
    return res

I, J = 500, 500
%time f_py(I, J)

import pyximport
pyximport.install()

# DOES NOT WORK !!!!!!!!!
# REST IN THIS SECTION IS SKIPPED

from nested_loop import f_cy

%time res = f_cy(I, J)

res

%load_ext cythonmagic

%%cython
#
# Nested loop example with Cython
#
def f_cy(int I, int J):
    cdef double res = 0
    # double float much slower than int or long
    for i in range(I):
        for j in range (J * I):
            res += 1
    return res

%time res = f_cy(I, J)

res

import numba as nb

f_nb = nb.jit(f_py)

%time res = f_nb(I, J)

res

func_list = ['f_py', 'f_cy', 'f_nb']
I, J = 500, 500
data_list = 3 * ['I, J']

perf_comp_data(func_list, data_list, rep=1)